In [1]:
import cv2              
import numpy as np      
import os                 
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
train_dir = "/Users/harshjhunjhunwala/desktop/projects/deep_learning_cost_function/dataset/train"
test_dir = "/Users/harshjhunjhunwala/desktop/projects/deep_learning_cost_function/dataset/test"
img_size = 50

In [3]:
# Labelling the cat and dog images separately

def label_img(img):
    temp_label = img.split('.')[-3]
    if temp_label == 'cat':
        return [1, 0]
    elif temp_label == 'dog':
        return [0, 1]


In [4]:
# Create training_set

def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(train_dir)):
        label = label_img(img)
        path = os.path.join(train_dir, img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_size,img_size))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

train_data = create_train_data()

train_set_x_orig = []
for img in train_data:
    train_set_x_orig.append(img[0])

train_set_y = []
for img in train_data:
    train_set_y.append(img[1])


100%|██████████| 25000/25000 [01:56<00:00, 213.89it/s]


In [5]:
#Creating test_set

def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(test_dir)):
        path = os.path.join(test_dir,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (img_size,img_size))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

test_data = process_test_data()

test_set_x_orig = []
for img in test_data:
    test_set_x_orig.append(img[0])

test_set_y = []

 11%|█         | 1377/12500 [00:06<00:56, 198.46it/s]


KeyboardInterrupt: 

In [ ]:
# # Loading in the data (cat and dog images in .jpg)
# def load_dataset():
#     with h5py.File('datasets/train_catvnoncat.h5', "r") as train_dataset:
#         train_set_x_orig = np.array(train_dataset["train_set_x"][:])
#         train_set_y_orig = np.array(train_dataset["train_set_y"][:])

#     with h5py.File('datasets/test_catvnoncat.h5', "r") as test_dataset:
#         test_set_x_orig = np.array(test_dataset["test_set_x"][:])
#         test_set_y_orig = np.array(test_dataset["test_set_y"][:])
#         classes = np.array(test_dataset["list_classes"][:])

#     train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
#     test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

#     return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

# train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [5]:
# Creating the helper functions
# sigmoid function:

def sigmoid(x):
    s = 1/ (1 + np.exp(-x))
    return s

In [3]:
# Initialising parameters
# Creating vector with zeroes and initialising w and b param

def initialise(v):
    """
    Create a vector of np.shape() = (v, 1) and initialise b to 0
    """
    w = np.zeros((v, 1))
    b = 0 
    return w, b

In [1]:
# Forward and Backward Propogation

def propogation(w, b, X, Y):
    """
    Implement cost function and its gradient for the propogation
    """
    m = X.shape[1] # m is used to average out the cost function and so it is the total number of columns in X
    
    # Forward Propogation
    A = sigmoid(np.dot(w.T, X) + b)
    cost = (-1 / m) * np.sum(Y * np.log(A) + (1 - Y)* np.log(1 - A)) 
    
    # Backward Propogation
    dw = 1/m * np.dot(X, (A - Y).T)
    db = i/m * np.sum(A - Y)
    
    grads = {"dw": dw, "db": db}
    
    return grads, cost
    

In [2]:
# Optimization 

def optimize(w, b, X, Y, num_iter, learning_rate, print_cost):
    costs = []
    for i in range(num_iter):
        grads, cost = propogation(w, b, X, Y)
        dw = grads['dw']
        db = grads['db']
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        if i % 100 == 0:
            costs.append(cost)
        
        if print_cost and i % 100 == 0:
            print("Cost after iteration %i: %f" % (i, cost))
        params = {'w': w, 'b': b}
        grads = {"dw": dw, "db": db}

        return params, grads, costs    

In [3]:
# Building the initial prediction model using our new w and b

def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    # Vector A is to predict the probability
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
    
    return Y_prediction